In [2]:
# utilities
from typing import Sequence, Mapping


def evaluation(y_test, y_pred):

  return dict(
      accuracy=accuracy_score(y_test, y_pred),
      recall=recall_score(y_test, y_pred, average='weighted', zero_division=0),
      precision=precision_score(y_test, y_pred, average='weighted', zero_division=0),
      f1_score=f1_score(y_test, y_pred, average='weighted', zero_division=0),
  )


def pprint(data, start = '', end = '\r\n', output = False, tab = '\t'):
  if isinstance(data, Mapping):
    temp = '|' + end
    
    for k, v in data.items():
      temp += start + k + ' = ' + pprint(v, tab + start, end, output=True) + end
    
    if not output:
      print(temp)
      
    return temp

  if not isinstance(data, str):
    if isinstance(data, Sequence):
      temp = '|' + end
      
      for k, v in enumerate(data):
        temp += start + str(k) + ' = ' + pprint(v, tab + start, end, output=True) + end
      
      if not output:
        print(temp)
        
      return temp

  if not output:
    print(data)
    
  return str(data)

In [4]:
import joblib


with open('best_data_model.jbl', 'rb') as stream:
  best_data_model = joblib.load(stream)
  pprint(best_data_model, tab='  ')

|
name = RandomForestClassifier_x_smote
model = RandomForestClassifier(random_state=42)
use_smote = True
use_adasyn = False
preprocessing = |
  0 = SMOTE(random_state=42)

scores = |
  accuracy = 0.9085365853658537
  recall = 0.9085365853658537
  precision = 0.9107433217189315
  f1_score = 0.908558228819527


